# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
nltk.download(['punkt','wordnet','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessageToCategories',con=engine)
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values

In [85]:
classes = Y.columns
classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [48]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'], dtype=object)

In [86]:
Y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
len(X)

26216

In [52]:
len(Y)

26216

### 2. Write a tokenization function to process your text data

In [53]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [54]:
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 

Information about the National Palace-
['information', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'tent', 'water', 'we', 'silo', 'thank'] 

I would like to receive the messages, thank you
['i', 'would', 'like', 'receive', '

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidTransformer
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
                ('vect', CountVectorizer())
                ('tfidf', TfidfTransformer())
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                ])

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y) 

In [56]:

vect = CountVectorizer(tokenizer=tokenize)

In [57]:
X_train = vect.fit_transform(X_train)

In [58]:
dict = vect.vocabulary_
sorted(dict.items(), key=lambda kv: kv[1], reverse=True)

[('zz', 27626),
 ('zwaar', 27625),
 ('zuoji', 27624),
 ('zuevo', 27623),
 ('zucula', 27622),
 ('ztfbqjht', 27621),
 ('zrcs', 27620),
 ('zover', 27619),
 ('zordon', 27618),
 ('zopope', 27617),
 ('zoonotic', 27616),
 ('zoonosis', 27615),
 ('zoomed', 27614),
 ('zoom', 27613),
 ('zoo', 27612),
 ('zoning', 27611),
 ('zongon', 27610),
 ('zongo', 27609),
 ('zoneb', 27608),
 ('zonea', 27607),
 ('zone', 27606),
 ('zonal', 27605),
 ('zona', 27604),
 ('zon', 27603),
 ('zomgggg', 27602),
 ('zombie', 27601),
 ('zodiac', 27600),
 ('zobas', 27599),
 ('zo', 27598),
 ('zn', 27597),
 ('zmyilk4d', 27596),
 ('zmir', 27595),
 ('zll8ca9pthe', 27594),
 ('zjctkaifpeople', 27593),
 ('ziziphus', 27592),
 ('zixi', 27591),
 ('zitig', 27590),
 ('ziti', 27589),
 ('zipingpu', 27588),
 ('zinc', 27587),
 ('zimbabwean', 27586),
 ('zimbabwe', 27585),
 ('zillion', 27584),
 ('zilla', 27583),
 ('zila', 27582),
 ('zijn', 27581),
 ('zijin', 27580),
 ('zigzacles', 27579),
 ('ziarat', 27578),
 ('zhuhai', 27577),
 ('zhuang', 27

In [59]:
sorted(dict.items(), key=lambda kv: kv[1])

[('0', 0),
 ('00', 1),
 ('000', 2),
 ('0000', 3),
 ('000ha', 4),
 ('000l', 5),
 ('000lt', 6),
 ('000ltrs', 7),
 ('000m2', 8),
 ('004', 9),
 ('00h58', 10),
 ('01', 11),
 ('010', 12),
 ('017', 13),
 ('019', 14),
 ('01st', 15),
 ('02', 16),
 ('020', 17),
 ('024', 18),
 ('027', 19),
 ('03', 20),
 ('030', 21),
 ('0300', 22),
 ('0310', 23),
 ('034', 24),
 ('035', 25),
 ('0350', 26),
 ('039', 27),
 ('04', 28),
 ('040', 29),
 ('0400', 30),
 ('043', 31),
 ('048', 32),
 ('04m', 33),
 ('05', 34),
 ('050', 35),
 ('0500', 36),
 ('052', 37),
 ('053', 38),
 ('056', 39),
 ('05am', 40),
 ('06', 41),
 ('063', 42),
 ('065', 43),
 ('066', 44),
 ('06f98jo5jersey', 45),
 ('07', 46),
 ('071', 47),
 ('077', 48),
 ('07th', 49),
 ('08', 50),
 ('081fmhg8', 51),
 ('082', 52),
 ('09', 53),
 ('090', 54),
 ('093', 55),
 ('0h4z', 56),
 ('0tranbleman', 57),
 ('0veupw3p', 58),
 ('0wbiqkj3', 59),
 ('1', 60),
 ('10', 61),
 ('100', 62),
 ('1000', 63),
 ('1000gdes', 64),
 ('1003', 65),
 ('100g', 66),
 ('100gd', 67),
 ('100

In [60]:

tfidf = TfidfTransformer()

In [61]:
X_train_tfidf = tfidf.fit_transform(X_train)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [88]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
model = MultiOutputClassifier(clf)
model.fit(X_train_tfidf,Y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [89]:
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

In [90]:
Y_pred = model.predict(X_test_tfidf)

In [91]:
from sklearn.metrics import classification_report, accuracy_score

In [92]:
#print(classification_report(Y_test[1],Y_pred[1],labels=36, target_names=classes))
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=classes))

                        precision    recall  f1-score   support

               related       0.16      0.09      0.12      1119
               request       0.00      0.00      0.00        26
                 offer       0.40      0.33      0.36      2708
           aid_related       0.09      0.05      0.06       505
          medical_help       0.03      0.02      0.02       315
      medical_products       0.01      0.01      0.01       182
     search_and_rescue       0.02      0.01      0.01       106
              security       0.03      0.01      0.02       229
              military       0.00      0.00      0.00         0
           child_alone       0.05      0.03      0.04       425
                 water       0.08      0.05      0.06       751
                  food       0.08      0.05      0.06       565
               shelter       0.00      0.00      0.00        95
              clothing       0.06      0.03      0.04       180
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [95]:
print("         Accuracy Score: ")
print("         ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))

         Accuracy Score: 
         ---------------
                  related : 0.65
                  request : 0.76
                    offer : 0.99
              aid_related : 0.52
             medical_help : 0.89
         medical_products : 0.92
        search_and_rescue : 0.96
                 security : 0.98
                 military : 0.95
              child_alone : 1.00
                    water : 0.90
                     food : 0.83
                  shelter : 0.87
                 clothing : 0.98
                    money : 0.96
           missing_people : 0.98
                 refugees : 0.95
                    death : 0.93
                other_aid : 0.81
   infrastructure_related : 0.90
                transport : 0.93
                buildings : 0.92
              electricity : 0.97
                    tools : 0.99
                hospitals : 0.98
                    shops : 0.99
              aid_centers : 0.98
     other_infrastructure : 0.93
          weather_related

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.